<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/w2d1_updates/tutorials/W2D1_ConvnetsAndRecurrentNeuralNetworks/W2D1_Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 3: Introduction to RNNs
**Week 2, Day 1: Convnets And Recurrent Neural Networks**

**By Neuromatch Academy**

__Content creators:__ Dawn McKnight, Richard Gerum, Cassidy Pirlot, Rohan Saha, Liam Peet-Pare, Saeed Najafi, Alona Fyshe

__Content reviewers:__ Saeed Salehi, Lily Cheng, Yu-Fang Yang, Polina Turishcheva, Nina Kudryashova

__Content editors:__ Nina Kudryashova

__Production editors:__ Anmol Gupta, Spiros Chavlis 

*Based on material from:* Konrad Kording, Hmrishav Bandyopadhyay, Rahul Shekhar, Tejas Srivastava

**Our 2021 Sponsors, including Presenting Sponsor Facebook Reality Labs**

<p align='center'><img src='https://github.com/NeuromatchAcademy/widgets/blob/master/sponsors.png?raw=True'/></p>

---
# Tutorial Objectives
At the end of this tutorial, we will be able to:
- Understand the structure of a Recurrent Neural Network (RNN)
- Build a simple RNN model

 

In [1]:
# @title Tutorial slides

# @markdown These are the slides for the videos in this tutorial
from IPython.display import IFrame
IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/5asx2/?direct%26mode=render%26action=download%26mode=render", width=854, height=480)

---
# Setup

In [2]:
# @title Install dependencies
!pip install livelossplot --quiet
!pip install opencv-python --quiet
!pip install unidecode

In [3]:
# Imports
import time
import math
import torch
import random
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from time import sleep
from IPython.display import HTML
from torch.autograd import Variable
from tqdm.notebook import tqdm, trange

In [4]:
# @title Figure settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/content-creation/main/nma.mplstyle")

plt.rcParams["mpl_toolkits.legacy_colorbar"] = False

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

In [5]:
# @title Helper functions
# https://github.com/spro/char-rnn.pytorch
import unidecode
import string

# Reading and un-unicode-encoding data
all_characters = string.printable
n_characters = len(all_characters)

def read_file(filename):
  file = unidecode.unidecode(open(filename).read())
  return file, len(file)


# Turning a string into a tensor
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    try:
      tensor[c] = all_characters.index(string[c])
    except:
      continue
  return tensor


# Readable time elapsed
def time_since(since):
  s = time.time() - since
  m = math.floor(s / 60)
  s -= m * 60
  return '%dm %ds' % (m, s)

def generate(decoder, prime_str='A', predict_len=100, temperature=0.8,
             device='cpu'):

  hidden = decoder.init_hidden(1)
  prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

  hidden = hidden.to(device)
  prime_input = prime_input.to(device)
  predicted = prime_str

  # Use priming string to "build up" hidden state
  for p in range(len(prime_str) - 1):
    _, hidden = decoder(prime_input[:,p], hidden)

  inp = prime_input[:,-1]

  for p in range(predict_len):
    output, hidden = decoder(inp, hidden)

    # Sample from the network as a multinomial distribution
    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]

    # Add predicted character to string and use as next input
    predicted_char = all_characters[top_i]
    predicted += predicted_char
    inp = Variable(char_tensor(predicted_char).unsqueeze(0))
    inp = inp.to(device)

  return predicted

In [6]:
# @title Set random seed

# @markdown Executing `set_seed(seed=seed)` you are setting the seed

# for DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.
import random
import torch

def set_seed(seed=None, seed_torch=True):
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')


# In case that `DataLoader` is used
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

In [7]:
#@title Set device (GPU or CPU). Execute `set_device()`
# especially if torch modules used.

# inform the user if the notebook uses GPU or CPU.

def set_device():
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

In [8]:
SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()

Random seed 2021 has been set.
GPU is enabled in this notebook.


---
# Section 1: Recurrent Neural Networks (RNNs)

In [9]:
# @title Video 1: RNNs
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = "https://player.bilibili.com/player.html?bvid={0}&page={1}".format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id=f"BV1L44y1m7PP", width=854, height=480, fs=1)
  print("Video available at https://www.bilibili.com/video/{0}".format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id=f"PsZjS125lLs", width=854, height=480, fs=1, rel=0)
  print("Video available at https://youtube.com/watch?v=" + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

RNNs are compact models that operate over timeseries, and have the ability to remember past input. They also save parameters by using the same weights at every time step.  If you've heard of Transformers, those models don't have this kind of temporal weight sharing, and so they are *much* larger.

The code below is adapted from [this github repository](https://github.com/spro/char-rnn.pytorch).

In [10]:
# @title Run Me to get the data
!wget --output-document=/content/sample_data/twain.txt https://raw.githubusercontent.com/amfyshe/amfyshe.github.io/master/twain.txt

--2021-07-24 14:24:57--  https://raw.githubusercontent.com/amfyshe/amfyshe.github.io/master/twain.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1476181 (1.4M) [text/plain]
Saving to: ‘/content/sample_data/twain.txt’

/content/sample_dat 100%[===================>]   1.41M  --.-KB/s    in 0.05s   

2021-07-24 14:24:57 (27.5 MB/s) - ‘/content/sample_data/twain.txt’ saved [1476181/1476181]



In [11]:
# RNN
# https://github.com/spro/char-rnn.pytorch
class CharRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
    super(CharRNN, self).__init__()
    self.model = model.lower()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    self.encoder = nn.Embedding(input_size, hidden_size)
    if self.model == "gru":
      self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
    elif self.model == "lstm":
      self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
    elif self.model == "rnn":
      self.rnn = nn.RNN(hidden_size, hidden_size, n_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    batch_size = input.size(0)
    encoded = self.encoder(input)
    output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
    output = self.decoder(output.view(batch_size, -1))
    return output, hidden

  def init_hidden(self, batch_size):
    if self.model == "lstm":
      return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
              Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
    return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

This next section of code takes care of training the RNN on several of Mark Twain's books.  In this short section, we won't dive into the code, but you'll get to learn a lot more about RNNs in a few days!  For now, we are just going to observe the training process.

One cool thing about RNNs is that they can be used to _generate_ language based on what the network sees during training.  As the network makes predictions, instead of confirming of those predictions are correct against some training text, we just feed them back into the model as the next observed token.  Starting from a random vector for the hidden state, we can generate many original sentences!  And what the network generates will reflect the text it was trained on.

Let's try it!  Run the code below.  As the network trains, it will output samples of generated text every 25 epochs.  Notice that as the training progresses, the model learns to spell short words, then learns to string some words together, and eventually can produce meaningful sentences (sometimes)!  Keep in mind that this is a relatively small network, and doesn't employ some of the cool things you'll learn about later in the week (e.g. LSTMs, though you can change that in the code below by changing the value of the `model` variable if you wish!)

After running the model, and observing the output, get together with your pod, and talk about what you noticed during training.  Did your network produce anything interesting?  Did it produce anything characteristic of Twain?  

**Note:** training for the full 2000 epochs is likely to take a while, so you may need to stop it before it finishes.

**Important:** You can increase the number of epochs `n_epochs` to 2000.

In [12]:
# https://github.com/spro/char-rnn.pytorch
batch_size = 50
chunk_len = 200
model = "rnn"  # other options: `lstm`, `gru`

# hyperparams
n_layers = 2
hidden_size = 200
n_epochs = 1000  # initial was set to 2000
learning_rate = 0.01
print_every = 25

def train(inp, target, device='cpu'):
  hidden = decoder.init_hidden(batch_size)
  decoder.zero_grad()
  loss = 0

  for c in range(chunk_len):
    output, hidden = decoder(inp[:,c].to(device), hidden.to(device))
    loss += criterion(output.view(batch_size, -1), target[:,c])

  loss.backward()
  decoder_optimizer.step()
  return loss.item() / chunk_len


file, file_len = read_file('/content/sample_data/twain.txt')


def random_training_set(chunk_len, batch_size, device='cpu'):
  inp = torch.LongTensor(batch_size, chunk_len).to(device)
  target = torch.LongTensor(batch_size, chunk_len).to(device)
  for bi in range(batch_size):
    start_index = random.randint(0, file_len - chunk_len - 1)
    end_index = start_index + chunk_len + 1
    chunk = file[start_index:end_index]
    inp[bi] = char_tensor(chunk[:-1])
    target[bi] = char_tensor(chunk[1:])
  inp = Variable(inp)
  target = Variable(target)
  return inp, target


decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=model,
    n_layers=n_layers,
    ).to(DEVICE)

decoder_optimizer = torch.optim.Adagrad(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

print("Training for %d epochs..." % n_epochs)
for epoch in tqdm(range(1, n_epochs + 1), position=0, leave=True):
  loss = train(*random_training_set(chunk_len, batch_size, DEVICE), device=DEVICE)
  loss_avg += loss

  if epoch % print_every == 0:
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch,\
                                   epoch / n_epochs * 100, loss))
    print(f"{generate(decoder, prime_str='Wh', predict_len=100, device=DEVICE)}\n")

Training for 1000 epochs...


 6s (25 2%) 2.3255]
Whey
aid te meted
-
kitheran no withe; the lest sowere out pa ghis ale he lonn therind andersishe the 

 12s (50 5%) 2.1357]
Wher hig't and ae!
HeoCgect en that he to ed whe fon thenlethinl treade the she nullind. Jeare the nir

 19s (75 7%) 2.0616]
Where faed of a forthing the min't seaver heisspid toired whon
aght it theng then wisty, and_
way. I d

 25s (100 10%) 2.0059]
Wh. I ment the wherd of but her
to the parspain't wis in the witce in the somer mas povelight his him 

 31s (125 12%) 1.9504]
Whill the in to cometing, he had him;
uple was clike. I down, and hand her what wounded a gorther evea

 38s (150 15%) 1.9222]
Wh the was was eletth; and nestherch, Indy--their a at the here Jad thatodes. Thot gorinced, conepp an

 45s (175 17%) 1.9001]
Whered most the the werd the but the sintures, and when
a plain the kis
wast,
rest her we doy of then 

 51s (200 20%) 1.8487]
Whing, it betarn says:

"The correed see hore he ragles to go and _there a pine in wook it his 

Now you can generate more examples using a trained model. Recall that `generate` takes the mentioned below arguments arguments to work:

```python
generate(decoder, prime_str='A', predict_len=100, temperature=0.8, device='cpu')
```

Try it by yourself

In [13]:
print(f"{generate(decoder, prime_str='Wh', predict_len=100, device=DEVICE)}\n")

When me
is furned everybody over they want down an all the forcious or it was by interrided coments up



---
# Section 2: Power consumption in Deep Learning

Training NN models can be incredibly costly, both in actual money but also in power consumption. 

In [14]:
# @title Video 2: Carbon Footprint of AI
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = "https://player.bilibili.com/player.html?bvid={0}&page={1}".format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id=f"BV1My4y1j7HJ", width=854, height=480, fs=1)
  print("Video available at https://www.bilibili.com/video/{0}".format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id=f"as6C334LmRs", width=854, height=480, fs=1, rel=0)
  print("Video available at https://youtube.com/watch?v=" + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

Take a few moments to chat with your pod about the following points:
* Which societal costs of training do you find most compelling?
* When is training an AI model worth the cost?  Who should make that decision?
* Should there be additional taxes on energy costs for compute centers? 

## Exercise 2: Calculate the carbon footprint that your pod generated today.
You can use this [online calculator](https://mlco2.github.io/impact/#compute)

---
# Summary

What a day!  We've learned a lot!  The basics of CNNs and RNNs, and how changes to architecture that allow models to parameter share can greatly reduce the size of the model.  We learned about convolution and pooling, as well as the basic idea behind RNNs.  To wrap up we thought about the impact of training large NN models.